In [ ]:
import csv
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Scales the data before fitting the model

df = pd.read_csv("full-data.csv")

cat_variables = ["gender", "ethnicity", "major"]

for st in cat_variables:
    df[st] = df[st].astype("category")
    df[st] = df[st].cat.codes

if df.isnull().sum().sum() != 0:
    raise Exception("Data contains null values. Can't proceed with linear regression")
X = df.drop(columns=["sleep"])
y = df["sleep"]

lr = LinearRegression()

def fitdata(X,y):
    samples = X.shape[0]
    dof = X.shape[1]
    sts= StandardScaler()
    sts.fit(X,y)
    scaled_X = sts.transform(X)
    data_scaled = pd.DataFrame(scaled_X, columns=X.columns)
    lr.fit(scaled_X, y)
    c = lr.intercept_
    m = lr.coef_
    r2 = r2_score(y, lr.predict(scaled_X))
    adjr2 = 1 - (1 - r2) * (samples - 1) / (samples - dof - 1)
    with open('readings-coff.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(["Column name", "Coefficient"])
        for i in range(len(X.columns)):
            writer.writerow([X.columns[i], m[i]])
    return r2, adjr2

fitdata(X,y)
